# Generateur

## Environement de travail

### Library

In [1]:
import pandas as pd
import numpy as np

import os
from glob import glob
import cv2

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

2022-02-21 14:51:59.596503: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jchikhaoui/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-21 14:51:59.596544: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Path

In [2]:
path = "/mnt/c/Users/jchik/Documents/m2/challenge"
os.chdir(path)
!ls

data  data_exploration	generateur


## Extraction des données

In [83]:
label = [os.path.basename(label) for label in glob("data/dataset_lucas/train/*")]
img_link = np.array([[label_, img] for label_ in label for img in glob("data/dataset_lucas/train/" + label_ + "/*")])
df = pd.DataFrame(data = {"label": img_link[:, 0], "img_link": img_link[:, 1]})

In [84]:
def extract_img(link):
    img = cv2.imread(link)
    return img

In [85]:
df["img"] = df.img_link.map(extract_img)
df.head()

,label,img_link,img
0,1355955,data/dataset_lucas/train/1355955/14ca6849c6702...,"[[[121, 121, 133], [121, 121, 133], [121, 121,..."
1,1355955,data/dataset_lucas/train/1355955/186c3c54ac3ad...,"[[[99, 117, 124], [98, 116, 123], [97, 115, 12..."
2,1355955,data/dataset_lucas/train/1355955/1ada170b59229...,"[[[28, 40, 28], [27, 39, 27], [28, 39, 29], [3..."
3,1355955,data/dataset_lucas/train/1355955/2b00392197370...,"[[[52, 59, 132], [53, 60, 133], [53, 59, 134],..."
4,1355955,data/dataset_lucas/train/1355955/5103fe9f21342...,"[[[82, 76, 95], [81, 75, 94], [81, 75, 94], [8..."


### Generateur

In [5]:
SIZE = 200
BATCH = 32
BATCH_SIZE = 0

In [113]:
class Generator:
    def __init__(self, size = SIZE, batch = BATCH, machine = "GPU4"):
        self.size = size
        self.batch = batch
        self.path = self.path(machine)
        self.label_img = self.label_img()
        self.tirage = self.my_shuffle()
    
    def path(self, machine):
        if machine == "GPU4": 
            path = "/home/data/challenge_2022_miashs/train"
        else: 
            path = os.getcwd() + "/data/dataset_lucas/train"
            
        return path
            
    def label_img(self):
        label = [os.path.basename(label_) for label_ in glob(self.path + "/*")]
        label_img = [[label_, glob(self.path + "/" + label_ + "/*")] for label_ in label]
        
        return np.array(label_img, dtype= object)
    
    def my_shuffle(self):
        nbr_label = len(self.label_img)
        nbr_img = np.sum([len(li) for li in self.label_img[:, 1]])
        tirage_label = tf.random.uniform(
            shape = (nbr_img,),
            minval = 0,
            maxval = nbr_label,
            dtype = tf.dtypes.int32
        )
        tirage_img = tf.map_fn(
            fn = lambda x: tf.random.uniform(shape = (1,), minval = 0, maxval = len(self.label_img[x, 1]), dtype = tf.dtypes.int32),
            elems = tirage_label
        )
        
        return tirage_img, tirage_label

In [114]:
a = Generator(machine = "other")

In [115]:
tirage_img, tirage_label = a.tirage

In [116]:
tirage_label

<tf.Tensor: shape=(5356,), dtype=int32, numpy=array([ 0, 13, 16, ..., 11,  1,  1], dtype=int32)>

In [105]:
tf.map_fn(lambda x: len(a.label_img[x, 1]), a.tirage)

<tf.Tensor: shape=(5356,), dtype=int32, numpy=array([ 24,  46,  35, ...,   9, 873,  35], dtype=int32)>

In [100]:
def get_file(machine = "GPU4"):
    if machine == "GPU4": 
        path = "/home/data/challenge_2022_miashs/train"
    else: 
        path = os.getcwd() + "/data/dataset_lucas/train"
        
    label = [os.path.basename(label) for label in glob(path + "/*")]
    img_link = np.array(
        [[label_, img] for label_ in label for img in glob(path + "/" + label_ + "/*")]
    )
    return img_link


def read_image(path, label):
    img = tf.image.decode_jpeg(tf.io.read_file(path),  channels=3)
    img = tf.image.resize(img, (SIZE, SIZE))
    img = tf.cast(img, tf.int32)/255
    return img, label


def generator(x, y):
    
    # label tf
    # img tf
    # dataset
    
    # read img
    
    # shuffle
    
    # prefetch batch
    x = tf.constant(x)
    y = tf.constant(y)
    df = tf.data.Dataset.from_tensor_slices((x, y))
    df = df.map(read_image, num_parallel_calls=tf.data.AUTOTUNE).cache()
    df = df.prefetch(buffer_size=2).batch(BATCH, drop_remainder=True)
    return df

In [101]:
img_link

array([['1355955',
        'data/dataset_lucas/train/1355955/14ca6849c67020249a06f46ab28da8c41ffc6c28.jpg'],
       ['1355955',
        'data/dataset_lucas/train/1355955/186c3c54ac3ada38638c6c4dc0ea7c849701b486.jpg'],
       ['1355955',
        'data/dataset_lucas/train/1355955/1ada170b59229c5018c4a9a26a2c0173d2209105.jpg'],
       ...,
       ['1643349',
        'data/dataset_lucas/train/1643349/237c9130f482abfb82f497dd449aa23582dc27be.jpg'],
       ['1643349',
        'data/dataset_lucas/train/1643349/61c2cccf277ce06bcd907bb361a192575cd210ef.jpg'],
       ['1643349',
        'data/dataset_lucas/train/1643349/dc69748143921cbeb5bcdbcb4452c09948461fcc.jpg']],
      dtype='<U77')